In [2]:
import os 
import pygmsh
import gmsh
import meshio
import argparse
import numpy as np 

In [7]:
points = [
    [-1,-1,0],
    [-1, 1,0],
    [ 1, 1,0],
    [ 1,-1,0],
]

cells = [[0,1,2,3]]

inj_point_coor = [0,0,0]

res = 0.1

fracture_points = [
    [-0.5, 0],
    [ 0.5, 0],
    [-0.5, -0.5],
    [0.5,  0.5],
]

fractures = [
    [0,1],
    [2,3]
]


with pygmsh.occ.Geometry() as geom:

    # adding points
    points = [geom.add_point(point, res) for point in points]

    # adding lines
    loops = []
    # loop over cell type
    for cell in cells:
        nb_vertices = len(cell)
        lines = [geom.add_line(points[cell[i]],points[cell[(i+1)%nb_vertices]]) for i in range(nb_vertices)]
        loops.append(geom.add_curve_loop(lines))

    # adding surfaces
    surfaces = [geom.add_plane_surface(loop) for loop in loops]
    
    # adding lines = fractures
    fracture_points = [geom.add_point(point, res) for point in fracture_points]

    lines = []
    for fracture in fractures:
        lines.append(geom.add_line(fracture_points[fracture[0]],fracture_points[fracture[1]]))
        
    gmsh.model.mesh.setSize([l.dim_tag for l in lines], res)

    # Injection point
    # inj_point = geom.add_point(inj_point_coor, res)

    geom.synchronize()

    # Computing fragments between surfaces and injection point
    fragments = geom.boolean_fragments(surfaces, lines)

    geom.synchronize()

    # Getting the resulting surfaces and imposing mesh resolution at their boundaries
    new_surfaces = [elmt for elmt in fragments if elmt.dim_tag[0] == 2]
    new_surfaces_dim_tag = [gmsh_obj.dim_tag for gmsh_obj in new_surfaces]
    
    new_lines = [elmt for elmt in fragments if elmt.dim_tag[0] == 1]
    new_lines_dim_tag = [gmsh_obj.dim_tag for gmsh_obj in new_lines]
    
    gmsh.model.mesh.setSize(gmsh.model.getBoundary(new_surfaces_dim_tag + new_lines_dim_tag, False, False, True),res)
    
    geom.synchronize()
    
    geom.save_geometry("test_mesh_fractures_bulk_geom.vtk")

    # geom.set_recombined_surfaces(surfaces)
    
    # gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 0)
    
    g_mesh = geom.generate_mesh(order=1,algorithm=2)

# only keep elements of dim 2
g_mesh.cells = [cells for cells in g_mesh.cells if cells.dim == 2 ]

g_mesh.write(f"test_mesh_fractures_bulk.vtk")
g_mesh

Info    : Writing 'test_mesh_fractures_bulk_geom.vtk'...                                                                            
Info    : Done writing 'test_mesh_fractures_bulk_geom.vtk'


<meshio mesh object>
  Number of points: 589
  Number of cells:
    triangle: 1096